In [1]:
import pandas as pd
import numpy as np

In [2]:
# 读取数据集
df1 = pd.read_excel('./示例文件.xlsx',sheet_name='data1')
df1

,orderID,customerID,order_date,store,product,quantity,price
0,O1,C1,2020-01-09,A,A,1,100
1,O1,C1,2020-01-09,A,B,1,300
2,O1,C1,2020-01-09,A,C,1,200
3,O2,C2,2020-02-01,B,B,1,300
4,O2,C2,2020-02-01,B,C,1,200
5,O3,C3,2020-03-11,A,A,1,100
6,O3,C3,2020-03-11,A,D,1,150
7,O3,C3,2020-03-11,A,S,1,500
8,O4,C4,2020-03-15,A,B,1,300
9,O4,C4,2020-03-15,A,F,2,700


In [3]:
df2 = pd.read_excel('./示例文件.xlsx',sheet_name='data2')
df2

,product,category,function,retail_price
0,A,A,A,100
1,B,B,A,300
2,C,C,A,200
3,D,A,B,150
4,F,A,C,350
5,S,C,B,200


In [4]:
df3 = pd.read_excel('./示例文件.xlsx',sheet_name='data3')
df3

,store,city,region
0,A,A,A
1,B,B,B
2,C,A,A
3,D,A,A
4,E,C,B


In [5]:
# 计算 2020 年第 1 季度的购买人数、销售金额、客单价、客单件、人均购买频次
con1 = df1['order_date'] >= '2020-01-01'
con2 = df1['order_date'] <= '2020-03-31'
df = df1[con1 & con2]
count_customer = len((df['customerID'].unique()))
count_order = len((df['orderID'].unique()))
price_sum = df['price'].sum()
quantity_sum = df['quantity'].sum()
price_per_order = price_sum / count_order
quantity_per_order = quantity_sum / count_order
times_per_customer = count_order / count_customer
print(f'2020 年第 1 季度的购买人数有 {count_customer} 人，销售金额有 {price_sum} 元，客单价为 {price_per_order} 元/单，客单件为 {quantity_per_order} 件/单，人均购买频次为 {times_per_customer} 次/人')

2020 年第 1 季度的购买人数有 5 人，销售金额有 3250 元，客单价为 650.0 元/单，客单件为 2.6 件/单，人均购买频次为 1.0 次/人


In [6]:
# 计算品牌在 2019-05 至 2020-04 期间的复购率
# 复购率 = 期间购买 2 比及以上的人数占比
con1 = df1['order_date'] >= '2019-05-01'
con2 = df1['order_date'] <= '2020-04-30'
df = df1[con1 & con2]
count_customer = len((df['customerID'].unique()))
df = df[['customerID','order_date']]
df_group1 = df.groupby(['customerID','order_date']).count().reset_index()
df_group2 = df_group1.groupby(['customerID'])['order_date'].count().reset_index()
df_group2 = df_group2[df_group2['order_date'] >= 2]
count_customer_gn_2 = len((df_group2['customerID'].unique()))
rate = str(round((count_customer_gn_2 / count_customer) *100,0)) + '%'
print(f'2019-05 至 2020-04 期间的复购率为 {rate}')

2019-05 至 2020-04 期间的复购率为 0.0%


In [7]:
# 给到每个城市的店铺数量和生意汇总，输出无购买记录的城市
df_merge = pd.merge(df1,df3,on='store',how='right')
df_merge = df_merge.fillna('')
df_merge = df_merge[df_merge['quantity'] == '']
df_merge = df_merge[['store','city','region']]
df_merge

,store,city,region
11,D,A,A
12,E,C,B


In [8]:
# 查找买过商品 B 和 C ，但没有买过商品 A 的顾客人数，并计算平均客单价
df = df1[['customerID','product']]
df_group = df.groupby(['customerID','product']).count().reset_index()
con1 = df_group['product'] == 'A'
con2 = df_group['product'] == 'B'
con3 = df_group['product'] == 'C'
df_group = df_group[con1 | con2 | con3]
df_pivot = pd.pivot_table(df_group,index='customerID',columns='product',values='product',aggfunc=len).fillna(0).astype(int)
con1 = df_pivot['A'] == 0
con2 = df_pivot['B'] == 1
con3 = df_pivot['C'] == 1
df_filter = df_pivot[con1 & con2 & con3].reset_index()
df_filter = df_filter[['customerID']]
count_customer = len(df_filter['customerID'].unique())
df_merge = pd.merge(df_filter,df1,on='customerID',how='left')
sum_price = df_merge['price'].sum()
price_per_customer = sum_price / count_customer
print(f'买过商品 B 和 C ，但没有买过商品 A 的顾客人数共有 {count_customer} 人，平均客单价是 {price_per_customer} 元/人')

买过商品 B 和 C ，但没有买过商品 A 的顾客人数共有 1 人，平均客单价是 500.0 元/人


In [9]:
# 查找每个城市购买金额排名第二的客人，列出其购买城市、姓名、购买金额
df_merge = pd.merge(df1,df3,on='store',how='right')
df_merge = df_merge.dropna()
df_group = df_merge.groupby(['city','customerID'])['price'].sum().reset_index()
df_group['组内降序排名'] = df_group.groupby(['city']).rank(method='dense',ascending=False).astype(int)
df_filter = df_group[df_group['组内降序排名'] == 2]
df_filter

,city,customerID,price,组内降序排名
1,A,C3,750.0,2
